In [ ]:
! pip install wget
! pip install azure-search-documents  
! pip install azure-identity
! pip install openai

In [ ]:
from azure.search.documents import IndexingError

In [ ]:
#  
# from azure.search.documents.indexes.models import IndexingError
# Try this alternative path if the direct import fails
from azure.search.documents._data_index_sender import IndexingError
 
# from azure.search.documents import IndexingError
from azure.search.documents import SearchIndexingBufferedSender

In [13]:
import json  
import wget
import pandas as pd
import zipfile
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryType,
    VectorizedQuery,
)
from azure.search.documents.indexes.models import (
    HnswAlgorithmConfiguration,
    HnswParameters,
    SearchField,
    SearchableField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)


Configure OpenAI settings

In [14]:
endpoint: str = "https://yy-foundry-test.openai.azure.com/"
api_key: str = "8yitPnw5we1beFK4nPEq6Jo8j0yNJbWkCPtlXiyhSLgJ5gjwDFMGJQQJ99BKAC4f1cMXJ3w3AAAAACOG3JcZ"
api_version: str = "2024-02-01"
deployment = "text-embedding-3-small"
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)

# Set this flag to True if you are using Azure Active Directory
use_aad_for_aoai = False 

if use_aad_for_aoai:
    # Use Azure Active Directory (AAD) authentication
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=api_version,
        azure_ad_token_provider=token_provider,
    )
else:
    # Use API key authentication
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        azure_endpoint=endpoint,
    )

In [15]:
# Configuration Azure AI Search Vector Store settings
search_service_endpoint: str = "https://ai-resource-test.search.windows.net"
search_service_api_key: str = "S888hymM1fzCiusn6upcfIXkWEETKhihqa41xZrusOAzSeAgfb5w"
index_name: str = "azure-ai-search-openai-cookbook-demo"

# Set this flag to True if you are using Azure Active Directory
use_aad_for_search = False  

if use_aad_for_search:
    # Use Azure Active Directory (AAD) authentication
    credential = DefaultAzureCredential()
else:
    # Use API key authentication
    credential = AzureKeyCredential(search_service_api_key)

# Initialize the SearchClient with the selected authentication method
search_client = SearchClient(
    endpoint=search_service_endpoint, index_name=index_name, credential=credential
)

In [ ]:
from azure.storage.blob import BlobServiceClient
import os
from  io import StringIO

def download_blob_to_file(blob_service_client: BlobServiceClient, container_name: str, blob_name: str):
       # Get the blob client
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    
    download_stream = blob_client.download_blob()
    blob_content = download_stream.readall().decode('utf-8')
    return StringIO(blob_content)

   # Example usage
connection_string = 'DefaultEndpointsProtocol=https;AccountName=yyaistorage;AccountKey=JmvI6Xzm8Ov2xeRgnpH43HjgNoch5zFaDDlaVlM5aziilZf00VcqXUUENfMKi5p535IVEEqMh0JW+ASt122zGQ==;EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = "yycontainertest"
blob_name = "vector_database_wikipedia_articles_embedded_sample.csv"
df = pd.read_csv(download_blob_to_file(blob_service_client, container_name, blob_name))
df.head()
   

In [58]:
def generate_embeddings(text, model):
    # Generate embeddings for the provided text using the specified model
    embeddings_response = client.embeddings.create(model=model, input=text)
    # Extract the embedding data from the response
    # embedding = embeddings_response.data[0].embedding
    embedding = embeddings_response.data[0].embedding
    return embedding

# why is data[0]

In [20]:
# df["id"] =  df["id"].astype(str)
# df["vector_id"] =  df["vector_id"].astype(str)
df["new_content_vector"] =  df["text"].apply(lambda x: generate_embeddings(x, deployment))
# Convert the DataFrame to a list of dictionaries
# documents =  df.to_dict(orient="records")

In [21]:
df.to_csv("vector_database_wikipedia_articles_embedded_sample_with_new_content_vector.csv", index=False)

In [ ]:
# Read vectors from strings back into a list using json.loads
df["title_vector"] =  df["title_vector"].astype(str)
df["content_vector"] =  df["content_vector"].astype(str)
df["new_content_vector"] =  df["new_content_vector"].astype(str)
df["title_vector"] = df.title_vector.apply(json.loads)
df["content_vector"] = df.content_vector.apply(json.loads)
df["new_content_vector"] = df.new_content_vector.apply(json.loads)

df.head()

In [ ]:
print(df["new_content_vector"][0])
print(df["content_vector"][0])

In [52]:
# Initialize the SearchIndexClient
index_client = SearchIndexClient(
    endpoint=search_service_endpoint, credential=credential
)
# Define the fields for the index
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String),
    SimpleField(name="vector_id", type=SearchFieldDataType.String, key=True),
    SimpleField(name="url", type=SearchFieldDataType.String),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="text", type=SearchFieldDataType.String),
    SearchField(
        name="title_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=1536,
        vector_search_profile_name="my-vector-config",
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=1536,
        vector_search_profile_name="my-vector-config",
    ),
    SearchField(
        name="new_content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=1536,
        vector_search_profile_name="my-vector-config",
    ),
]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="my-hnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE,
            ),
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="my-vector-config",
            algorithm_configuration_name="my-hnsw",
        )
    ],
)

# Configure the semantic search configuration
semantic_search = SemanticSearch(
    configurations=[
        SemanticConfiguration(
            name="my-semantic-config",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="title"),
                keywords_fields=[SemanticField(field_name="url")],
                content_fields=[SemanticField(field_name="text")],
            ),
        )
    ]
)

# Create the search index with the vector search and semantic search configurations
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)

# Create or update the index
result = index_client.create_or_update_index(index)
print(f"{result.name} created")

azure-ai-search-openai-cookbook-demo created


In [ ]:
df.head()

In [ ]:
df["id"] =  df["id"].astype(str)
df["vector_id"] =  df["vector_id"].astype(str)
documents =  df.to_dict(orient="records")
print(documents[0])

In [ ]:
#indexing_results = search_client.upload_documents(documents=documents[0])

In [54]:
from azure.core.exceptions import HttpResponseError
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents.models import IndexingResult


batch_client = SearchIndexingBufferedSender(
    search_service_endpoint, index_name, credential
)

try:
    # Add upload actions for all documents in a single call
    batch_client.upload_documents(documents=documents)

    # Manually flush to send any remaining documents in the buffer
    indexing_results = batch_client.flush(timeout=60)

except HttpResponseError as e:
    print(f"An error occurred: {e}")
finally:
    # Clean up resources
    batch_client.close()

print(f"Uploaded {len(documents)} documents in total")

Uploaded 100 documents in total


In [59]:
# Pure Vector Search
query = "modern art in Europe"
  

vector_query = VectorizedQuery(vector=generate_embeddings(query, deployment), k_nearest_neighbors=3, fields="new_content_vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query], 
    select=["title", "text", "url"] 
)
  
for result in results:
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"URL: {result['url']}\n")  

Title: Art
Score: 0.632367
URL: https://simple.wikipedia.org/wiki/Art

Title: Architecture
Score: 0.59899247
URL: https://simple.wikipedia.org/wiki/Architecture

Title: Austria
Score: 0.5748276
URL: https://simple.wikipedia.org/wiki/Austria



In [ ]:
# Semantic Hybrid Search
query = "What were the key technological advancements during the Industrial Revolution?"

search_client = SearchClient(search_service_endpoint, index_name, credential)
vector_query = VectorizedQuery(
    vector=generate_embeddings(query, deployment),
    k_nearest_neighbors=3,
    fields="content_vector",
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["title", "text", "url"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="my-semantic-config",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3,
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"URL: {result['url']}")
    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")